In [16]:
import sys

sys.path.append('/workspaces/uniswap-v3-crawler')

In [17]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import networkx as nx
import subprocess
import itertools as it
from ast import literal_eval
from src import graph, uniswap, sssp, uniprice, uniprice_2

In [18]:
def safe_literal_eval(val):
    try:
        if type(val) is dict:
            return val
        
        return literal_eval(val)
    except Exception:
        return None


In [19]:
import pandas as pd

df = pd.read_csv("../data/snapshots/snapshot_22688136.csv.gz")
df['other'] = df['other'].apply(safe_literal_eval)
df['token0Price'] = pd.to_numeric(df['token0Price'], errors='coerce')
df['token1Price'] = pd.to_numeric(df['token1Price'], errors='coerce')
df = df.dropna(subset=['other', 'token0Price', 'token1Price'])


# Check if 'feeTier' column exists, if not, create it
if 'feeTier' not in df.columns:
    df['feeTier'] = df.apply(
        lambda row: int(row['other'].get('feeTier', 0)) if row['version'] in ['v3', 'v4'] 
        else 3000 if row['version'] == 'v2' 
        else None, 
        axis=1
    )
    df['feeTier'] = df['feeTier'].apply(float)

        
df.head()

,createdAtBlockNumber,id,liquidity,token0Price,token1Price,totalValueLockedUSD,other,token0_decimals,token0_id,token0_name,token0_symbol,token1_decimals,token1_id,token1_name,token1_symbol,version,block_number,feeTier
0,22642921,0xd0bdf5e0da79d3f047a741fae43f0de12dc07fcdf924...,10375352384363653388,0.009854,101.482245,18.420975,"{'feeTier': '10000', 'sqrtPrice': '79813180273...",18,0x514910771af9ca656af840dff83e8264ecf986ca,ChainLink Token,LINK,18,0xf94e7d0710709388bce3161c32b4eea56d3f91cc,Destra Network,DSync,v4,22688136,10000.0
1,21880902,0xd185908c3b4f76d150a59b35be638daaff269eb66ac1...,6639386180484986,1.024226,0.976347,18.112682,"{'feeTier': '100', 'sqrtPrice': '7828558298452...",18,0x0000000000000000000000000000000000000000,Ethereum,ETH,18,0x0bfda9a1914dd56313da138143e871c2b7a01e39,STARFORCEONE GOLD,STFMG,v4,22688136,100.0
2,22161234,0x470571da0067d12e63b3a32eb1a4b220704e637c3e7c...,2291597475,1.058882,0.944392,17.790441,"{'feeTier': '10000', 'sqrtPrice': '76993790429...",8,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC,18,0x2f913c820ed3beb3a67391a6eff64e70c4b20b19,Merlin BTC,M-BTC,v4,22688136,10000.0
3,22161525,0xd720d4a7aa64f1e150418a5dffde50dc724cb84575a8...,2284397483,1.061188,0.942340,17.752730,"{'feeTier': '100', 'sqrtPrice': '7691009414817...",8,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC,18,0x2f913c820ed3beb3a67391a6eff64e70c4b20b19,Merlin BTC,M-BTC,v4,22688136,100.0
4,22263523,0xbc1537b776704384fcffb2514aa338fad53dcceff1be...,1795079231028376446,0.000031,32255.144927,17.626592,"{'feeTier': '500', 'sqrtPrice': '1422915409270...",18,0x0000000000000000000000000000000000000000,Ethereum,ETH,18,0x7de47cd8ea2feef2d1b1b720c3141601b2a5f404,Copein,COPEIN,v4,22688136,500.0


In [20]:
df_top_tokens = uniswap.get_top_tokens(df, 1000)
df_top_tokens.head()

,token_id,token_symbol
9747,0xd286511359a0b7f3d77d96a4fce06a9226a24353,unknown
7306,0x9cc32f63850a3d9a64d613a4fa7b1c7d8993c054,CRYPT
3919,0x5483dc6abda5f094865120b2d251b5744fc2ecb5,TPAD
7026,0x96884fcaac082db4b32601ada5b177fd6cbffa88,ZKLK
11331,0xf7498c98789957f4ee53b3e37ff5b7ef8a6cfc7b,0xDEV


In [21]:
coin_list = [
    "0x0000000000000000000000000000000000000000",
    "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
    "0x2260fac5e5542a773aa44fbcfedf7c193bc2c599", 
    "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", 
    "0xdac17f958d2ee523a2206206994597c13d831ec7"
    ]

In [22]:
token_set = set(df_top_tokens['token_id'].tolist()+coin_list)
df = df[df['token0_id'].isin(token_set) & df['token1_id'].isin(token_set)]
len(df)

2557

In [23]:
def create_graph_fixed(df, apply_fee=False, remove_low_degree_nodes=False, remove_multi_edges=False, fake_fee=0.0, remove_zero_reserves=False):
    # Create a graph
    if remove_multi_edges:
        G = nx.DiGraph()
    else:
        G = nx.MultiDiGraph()

    if apply_fee:
        df['token0Price'] = df['token0Price'] * (1 - df['feeTier'] / 1e6)
        df['token1Price'] = df['token1Price'] * (1 - df['feeTier'] / 1e6)

    if fake_fee > 0.0:
        df['token0Price'] = df['token0Price'] * (1 - fake_fee)
        df['token1Price'] = df['token1Price'] * (1 - fake_fee)

    for _, row in df.iterrows():
        # Add nodes
        G.add_node(row['token0_id'], token_symbol=row['token0_symbol'])
        G.add_node(row['token1_id'], token_symbol=row['token1_symbol'])

        # token1 -> token0
        weight_0 = math.log(row['token0Price']) * -1 if float(row['token0Price']) > 0 else 1e10
        if remove_multi_edges:
            if G.has_edge(row['token1_id'], row['token0_id']):
                existing_weight = G[row['token1_id']][row['token0_id']]['weight']
                if weight_0 < existing_weight:
                    G[row['token1_id']][row['token0_id']]['weight'] = weight_0
            else:
                G.add_edge(row['token1_id'], row['token0_id'], weight=weight_0, version=row.get('version', None))
        else:
            G.add_edge(row['token1_id'], row['token0_id'], weight=weight_0, version=row.get('version', None))

        # token0 -> token1
        weight_1 = math.log(row['token1Price']) * -1 if float(row['token1Price']) > 0 else 1e10
        if remove_multi_edges:
            if G.has_edge(row['token0_id'], row['token1_id']):
                existing_weight = G[row['token0_id']][row['token1_id']]['weight']
                if weight_1 < existing_weight:
                    G[row['token0_id']][row['token1_id']]['weight'] = weight_1
            else:
                G.add_edge(row['token0_id'], row['token1_id'], weight=weight_1, version=row.get('version', None))
        else:
            G.add_edge(row['token0_id'], row['token1_id'], weight=weight_1, version=row.get('version', None))

    # Remove nodes with degree less than 2 if the flag is set
    if remove_low_degree_nodes:
        while True:
            nodes_to_remove = [node for node in G.nodes() if G.in_degree(node) == 1 and G.out_degree(node) == 1]
            if len(nodes_to_remove) == 0:
                break
            G.remove_nodes_from(nodes_to_remove)

    return G


In [24]:
G = create_graph_fixed(df, apply_fee=True, remove_low_degree_nodes=False, remove_multi_edges=True, fake_fee=0.0, remove_zero_reserves=False)
len(G.nodes), len(G.edges)

(1000, 3556)

In [25]:
source = coin_list[0]

In [26]:
# plt.figure(figsize=(8, 6))

# # Use token symbols as node labels
# labels = {n: G.nodes[n].get('token_symbol', n) for n in G.nodes}

# # Layout for better visualization
# pos = nx.spring_layout(G, seed=42)

# # Draw nodes with labels
# nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)
# nx.draw_networkx_labels(G, pos, labels, font_size=8)

# # Draw directed edges with arrows
# nx.draw_networkx_edges(
#     G, pos, 
#     arrowstyle='->', arrowsize=20, 
#     connectionstyle='arc3,rad=0.15', 
#     edge_color='gray'
# )

# # Draw edge weights (labels)
# edge_labels = {(u, v): f"{d['weight']:.6f}" for u, v, d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(
#     G, pos, edge_labels=edge_labels, 
#     font_color='red', font_size=8, 
#     label_pos=0.65,  # closer to the target node
#     rotate=True
# )

# plt.axis('off')
# plt.title("Token Graph (Directed, with Coin Symbols and Edge Weights)")
# plt.show()

In [27]:
# # Find the negative cycle
# neg_cycle = nx.find_negative_cycle(G, coin_list[0])

# # Build a set of edges in the negative cycle (as (u, v) tuples)
# neg_cycle_edges = set()
# for i in range(len(neg_cycle)):
#     u = neg_cycle[i]
#     v = neg_cycle[(i + 1) % len(neg_cycle)]
#     neg_cycle_edges.add((u, v))

# plt.figure(figsize=(8, 6))

# # Use token symbols as node labels
# labels = {n: G.nodes[n].get('token_symbol', n) for n in G.nodes}

# # Use the same layout as before for consistency
# # (pos is already defined in previous cell)

# # Draw nodes
# nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)
# nx.draw_networkx_labels(G, pos, labels, font_size=8)

# # Draw all edges, coloring those in the negative cycle differently
# edge_colors = []
# for u, v in G.edges():
#     if (u, v) in neg_cycle_edges:
#         edge_colors.append('red')
#     else:
#         edge_colors.append('gray')

# nx.draw_networkx_edges(
#     G, pos,
#     edge_color=edge_colors,
#     arrowstyle='->', arrowsize=20,
#     connectionstyle='arc3,rad=0.15'
# )

# # Draw edge weights (labels)
# edge_labels = {(u, v): f"{d['weight']:.2f}" for u, v, d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(
#     G, pos, edge_labels=edge_labels,
#     font_color='red', font_size=8,
#     label_pos=0.65,
#     rotate=False
# )

# plt.axis('off')
# plt.title("Token Graph with Negative Cycle Highlighted")
# plt.show()

In [28]:
def bellman_ford_all_iterations(G, source, weight="weight"):
    """
    Bellman-Ford algorithm that does NOT stop when a negative cycle is found.
    Returns predecessors, distances, negative_edges, and a boolean indicating if a negative cycle exists.
    """
    if source not in G:
        raise nx.NodeNotFound(f"Node {source} is not found in the graph")

    # Weight function
    if callable(weight):
        weight_fn = weight
    else:
        def weight_fn(u, v, d):
            return d.get(weight, 1)

    # Initialization
    dist = {node: float("inf") for node in G.nodes}
    pred = {node: [] for node in G.nodes}
    dist[source] = 0

    # Relax edges |V|-1 times
    for _ in range(len(G.nodes) - 1):
        for u, v, d in G.edges(data=True):
            w = weight_fn(u, v, d)
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                pred[v] = [u]
            elif dist[u] + w == dist[v]:
                pred[v].append(u)

    # One more pass to check for negative cycles (optional, does not stop)
    negative_edges = []
    for u, v, d in G.edges(data=True):
        w = weight_fn(u, v, d)
        if dist[u] + w < dist[v]:
            negative_edges.append((u, v))

    has_negative_cycle = len(negative_edges) > 0

    return pred, dist, negative_edges, has_negative_cycle

pred, dist, negative_edges, has_negative_cycle = bellman_ford_all_iterations(G, coin_list[0])
has_negative_cycle

True

In [29]:
try:
    nx.bellman_ford_predecessor_and_distance(G, source)
except Exception as e:
    print(e)

Negative cycle detected.


In [30]:
up = uniprice_2.UniPrice()
up.accept_graph(G)

2025-06-25 08:49:16,121 - INFO - Treewidth: 10
2025-06-25 08:49:16,135 - DEBUG - Ratio new_edges/all_edges: 1.0359955005624297


2025-06-25 08:49:16,137 - DEBUG - Ratio new_edges/all_edges: 1.0359955005624297


{'singular_vertices': set(),
 'pre_singular_vertices': {'...'},
 'elimination_order': ['...']}

In [32]:
up.query_sssp(source)

({'0x514910771af9ca656af840dff83e8264ecf986ca': -11052939.141716266,
  '0xf94e7d0710709388bce3161c32b4eea56d3f91cc': -47791597.97645012,
  '0x9cdf242ef7975d8c68d5c1f5b6905801699b1940': -47596769.44771078,
  '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': -1490841.915771516,
  '0x0000000000000000000000000000000000000000': -97878.04588150377,
  '0x45804880de22913dafe09f4980848ece6ecbaf78': -4593684.766117033,
  '0x58b6a8a3302369daec383334672404ee733ab239': -50863250.49586157,
  '0xca14007eff0db1f8135f4c25b34de49ab0d42766': -51049495.34302164,
  '0x9ff58067bd8d239000010c154c6983a325df138e': -84600458.62350133,
  '0x738865301a9b7dd80dc3666dd48cf034ec42bdda': -47401992.671624266,
  '0xdac17f958d2ee523a2206206994597c13d831ec7': -2457886.5275034243,
  '0x39d5313c3750140e5042887413ba8aa6145a9bd2': -181582166.06350443,
  '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': -737265.7343741774,
  '0x6b4c7a5e3f0b99fcd83e9c089bddd6c7fce5c611': -47206159.42868659,
  '0x808507121b80c02388fad14726482e061b8da82